# Zharkbot In A RUST Shell
> Taking a look at this updated ZharkBot in a rust packer

- toc: true 
- badges: true
- categories: [zharkbot,rust,triage]

## Overview
Taking a look at this new variant of ZharkBot. It is delivered in a packer (currently unknown) which is written in rust. Based on the reporting from @0xperator on X ZharkBot appears to be a loader of sorts (to be confirmed).

![](https://pbs.twimg.com/media/GEnK0hFXEAA57Wg?format=png&name=4096x4096)

![](https://pbs.twimg.com/media/GEnK0g_WAAAApBe?format=png&name=large)



### References 
- https://x.com/0xperator/status/1750153123581083699
- https://x.com/ViriBack/status/1749184882822029564?s=20
- https://github.com/RussianPanda95/Yara-Rules/blob/main/ZharkBot/zharkbot.yar
- [Rust reversing workshop from @cxiao](https://github.com/cxiao/rust-reversing-workshop-recon-2024)
- [StackStack IDA plugin](https://github.com/idiom/stackstack)

### Samples
- Old Zharkbot (reference) `d53ce8c0a8a89c2e3eb080849da8b1c47eaac614248fc55d03706dd5b4e10bdd` [UnpacMe](https://www.unpac.me/results/9dc83159-5cfb-4e44-a3b8-588e1fba4d0c)
- New packed Zharkbot (rust packer) `068ef78225ab94c3f9c228d6248911986c23317d269f0bb5d0a46bd15cd93e80` [UnpacMe](https://www.unpac.me/results/06b086f9-f02c-4974-a821-9ea520acb32e)



## Analysis

### Rust Packer
The rust packer contains some anti-analysis features, and the ability to copy itself to `c:\Programdata\` and set persistence via the run key. In this sense it is possibly a bit more than a packer (packer plus persistence).

#### Build Strings
The developer has left the rust build strings in the binary which can be used to identify the libraries used (crates) and the developer's local username.

```
C:\Users\Magnu\.cargo\registry\src\index.crates.io-6f17d22bba15001f\antilysis-0.1.2\src\lib.rs
```

Anti analysis.
```
procexp64.exeprocexp.exeProcmon.exeProcmon64.exepestudio.exeKsDumper.exeprl_cc.exeprl_tools.exepe-sieve64.exeMoneta64.exefakenet.exeWireshark.exeVBoxService.exeVMwareUser.exevmtoolsd.exeVMwareTray.exevmsrvc.exeVBoxTray.execalled `Option::unwrap()` on a `None` value

C:\Users\Magnu\.cargo\registry\src\index.crates.io-6f17d22bba15001f\antilysis-0.1.2\src\lib.rs
```
Rust library to prevent windows programs from being analyzed. [link](https://docs.rs/crate/antilysis/latest/source/README.md)



Possible build path attributed to the developer.
```
C:\Users\Magnu\.cargo\registry\src\index.crates.io-6f17d22bba15001f\hex-0.4.3\src\lib.rs
```


### Anti Analysis

The packer employs a 30-second sleep which may be an anti-sandbox feature.

The first anti-analysis check searches for the following file, terminating the process if it is found. Based on the filename this is likely some sort of honeypot or sandbox but the exact origin is currently unknown.

```
C:\Users\admin\Documents\Outlook Files\honey@pot.com.pst
```

#### Antilysis
In addition to the honeypot check the packer uses the open source anti-analysis library [antilysis](https://docs.rs/crate/antilysis/latest/source/README.md). This library contains a simple process hostname check for `john-pc`, a check for the windows version equal to `0`, and a check for the following processes.

```
Wireshark.exe
procexp64.exe
procexp.exe
Procmon.exe
Procmon64.exe
pestudio.exe
KsDumper.exe
prl_cc.exe
prl_tools.exe
pe-sieve64.exe
hollows_hunter32.exe
Moneta64.exe
fakenet.exe
VBoxTray.exe
VBoxService.exe
VMwareUser.exe
vmtoolsd.exe
VMwareTray.exe
vmsrvc.exe
VGAuthService.exe
```

#### ETW and AMSI Patching
The packer patches out the userland ETW reporting APIs
```
EtwEvent1
EtwEventWrite
```

and patches out the AMSI reporting API
```
AMSIScanBuffer
```

### Payload
The payload is decrypted on the heap then loaded into executable memory using the standard VirtualAlloc and VirtualProtect APIs.




## ZharkBot

### Network

Old version c2
```
GET /zhark/api.php?id=bc1f01b7e0c4c374cdd60b3595677987&us=Admin&mn=VTILVGXH&os=Windows%207%20Ultimate&bld=1.1.0B HTTP/1.1
User-Agent: Mozilla/5.0(WindowsNT10.0;Win64;x64)AppleWebKit/537.36(KHTML,likeGecko)Chrome/74.0.3729.169Safari/537.36
```

New version c2
```
GET /api?id=0C9085685A4A8A525B8AADEAC0E44B9FB7804F8F07764BF463CC8E41397DBCF7&us=2FCD8A6551&mn=3EE2B1447274A028&os=39C08968505B98415E8FB59C9BF11E8FF1C744CD51&bld=1898C939111C HTTP/1.1
User-Agent: Mozilla/5.0(OpiumG4ng Win32)
```